In [12]:
# Bibliotecas

import pandas as pd
import inflection
import plotly.express as px 
from folium.plugins import MarkerCluster
from millify import millify as mil

# Leitura da base de dados

df = pd.read_csv("dataset/zomato.csv")

## Funções

In [4]:

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

def country_name(country_id):
    return COUNTRIES[country_id]

def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}

def color_name(color_code):
    return COLORS[color_code]

def cuisines_ratting(df, cuisine, ascending = True):
    cols = ['restaurant_name', 'country_name', 'aggregate_rating']
    df = df[df.cuisines.isin([cuisine])]
    df = df[cols]

    return df.sort_values('aggregate_rating', ascending= ascending)

## Entendimento dos Dados

In [31]:
# Shape dos dados
df.shape

(7527, 21)

In [4]:
# Descrição do dataframe
colums = ['Average Cost for two', 'Has Table booking','Has Online delivery', 'Price range']
df[colums].describe()

,Average Cost for two,Has Table booking,Has Online delivery,Price range
count,7.527000e+03,7527.000000,7527.000000,7527.000000
mean,7.152113e+03,0.062176,0.352464,2.624552
std,2.906068e+05,0.241491,0.477769,0.959794
min,0.000000e+00,0.000000,0.000000,1.000000
25%,6.000000e+01,0.000000,0.000000,2.000000
50%,2.900000e+02,0.000000,0.000000,3.000000
75%,6.000000e+02,0.000000,1.000000,3.000000
max,2.500002e+07,1.000000,1.000000,4.000000


In [5]:
# tipo de dados 
pd.DataFrame({'exemple':df.loc[0,:], 'type': df.dtypes})
#Restaurant ID -> object

,exemple,type
Restaurant ID,6310675,int64
Restaurant Name,Mama Lou's Italian Kitchen,object
Country Code,162,int64
City,Las Piñas City,object
Address,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",object
Locality,BF International,object
Locality Verbose,"BF International, Las Piñas City",object
Longitude,121.009787,float64
Latitude,14.447615,float64
Cuisines,Italian,object


In [13]:
df[df['Average Cost for two'] == 0].count()
df[df['Average Cost for two'] > 100000]

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
385,16608070,d'Arry's Verandah Restaurant,14,Adelaide,"Osborn Rd, McLaren Vale",McLaren Vale,McLaren Vale,138.545242,-35.198372,"Modern Australian, Australian",...,Dollar($),1,0,0,0,1,4.7,3F7E00,Excellent,203
6027,18474515,Kluwih Sunda Authentic,94,Bogor,"Jl. Binamarga I No. 12, Bogor Timur, Bogor",Bogor Timur,"Bogor Timur, Bogor",106.810896,-6.602734,Indonesian,...,Indonesian Rupiah(IDR),1,0,0,0,3,4.5,3F7E00,Excellent,573
6028,18428528,Foresthree,94,Bogor,"Jl. H. Achmad Adnawijaya, Bogor Utara, Bogor",Bogor Utara,"Bogor Utara, Bogor",106.817888,-6.577744,"Cafe, Western",...,Indonesian Rupiah(IDR),0,0,0,0,3,4.2,5BA829,Very Good,724
6029,18575783,Gyu - Kaku Japanese BBQ,94,Jakarta,"AEON Mall, Lantai Ground, Jl. Jakarta Garden C...","AEON Mall, Cakung","AEON Mall, Cakung, Jakarta",106.952958,-6.173134,"Japanese, BBQ",...,Indonesian Rupiah(IDR),0,0,0,0,3,4.7,3F7E00,Excellent,752
6030,7403971,Goedkoop,94,Jakarta,"Jl. Bendungan Hilir Raya No. 62, Bendungan Hil...",Bendungan Hilir,"Bendungan Hilir, Jakarta",106.812639,-6.212932,"Cafe, Coffee, Belanda",...,Indonesian Rupiah(IDR),0,0,0,0,2,4.3,5BA829,Very Good,891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,7417455,Talaga Sampireun,94,Jakarta,"Taman Impian Jaya Ancol, Jl. Lapangan Golf 7, ...","Taman Impian Jaya Ancol, Ancol","Taman Impian Jaya Ancol, Ancol, Jakarta",106.833353,-6.126930,"Sunda, Indonesian",...,Indonesian Rupiah(IDR),0,0,0,0,3,4.8,3F7E00,Excellent,3132
6105,18680113,The Cutt Grill House,94,Jakarta,"Jl. Tebet Barat Dalam Raya No. 23, Tebet, Jakarta",Tebet,"Tebet, Jakarta",106.846710,-6.236098,"Grill, Western",...,Indonesian Rupiah(IDR),1,0,0,0,3,4.8,3F7E00,Terbaik,675
6106,18615392,WAKI Japanese BBQ Dining,94,Jakarta,"Lantai 1, Jl. Tanjung Karang No.5, RT.11/RW.20...",Thamrin,"Thamrin, Jakarta",106.822396,-6.201092,"Japanese, Grill, BBQ, Korean, Asian",...,Indonesian Rupiah(IDR),1,0,0,0,3,4.9,3F7E00,Excellent,1242
6107,7425975,Seirock-Ya,94,Tangerang,"AEON Mall, Lantai 3, Ramen Village, Jl. BSD Ra...","AEON Mall, Serpong","AEON Mall, Serpong, Tangerang",106.642906,-6.305269,"Japanese, Ramen",...,Indonesian Rupiah(IDR),0,0,0,0,3,4.5,3F7E00,Excellent,798


## Limpeza dos dados

In [13]:
df1 = df.copy()

# 0. Mudar nome das colunas
df1 = rename_columns(df1)
df1
# 1. Transformar o tipo int para object na coluna "Restauran ID"
df1['restaurant_id'] = df1.loc[:,'restaurant_id'].astype(str)

#2 Remover valores ausentes nan
df1.dropna(inplace= True)

#3. categorizar por um tipo de culinaria
df1["cuisines"] = df1.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

#4. Remover valores duplicados
df1.drop_duplicates(inplace= True)

#5. Alterando variavel categorica price_range
df1['price_range'] = df1['price_range'].apply(lambda x: create_price_tye(x))

#6. Criando a coluna country_name
df1['country_name'] = df1['country_code'].map(COUNTRIES)

#7. Criando a coluna color_name
df1['rating_color_name'] = df1['rating_color'].map(COLORS)

#8. Removendo pratos que são iguais a zero
#df1 = df1[(df1['average_cost_for_two'] != 0) & (df1['aggregate_rating'] !=0)]


In [15]:
df1.sample(1).T

,4504
restaurant_id,18713367
restaurant_name,Rishi Vegetarian Dhaba NK
country_code,1
city,Ludhiana
address,"Sco - 88 Phase 1 Dugri, Urban Estate, Dugri"
locality,Dugri
locality_verbose,"Dugri, Ludhiana"
longitude,75.839964
latitude,30.874874
cuisines,North Indian


# Insigths

## H1 Restaurantes mais caros tem em média uma maior nota

In [21]:
aux = df1[['price_range', 'aggregate_rating']].groupby('price_range').mean().reset_index()
px.bar(aux, x = 'price_range', y = 'aggregate_rating',
        category_orders= {'price_range': ['cheap', 'normal', 'expensive', 'gourmet']})

## H2 Restaurantes que tem entregas Online recebem em média mais votos

In [28]:
aux = df1[['has_online_delivery', 'votes']].groupby('has_online_delivery').mean().reset_index()
px.bar(aux, x = 'has_online_delivery', y = 'votes')

In [26]:
df[df['Average Cost for two'] == df['Average Cost for two'].max()].T

,385
Restaurant ID,16608070
Restaurant Name,d'Arry's Verandah Restaurant
Country Code,14
City,Adelaide
Address,"Osborn Rd, McLaren Vale"
Locality,McLaren Vale
Locality Verbose,McLaren Vale
Longitude,138.545242
Latitude,-35.198372
Cuisines,"Modern Australian, Australian"


In [29]:
df1[df1['average_cost_for_two'] == df1['average_cost_for_two'].max()].T

,385
restaurant_id,16608070
restaurant_name,d'Arry's Verandah Restaurant
country_code,14
city,Adelaide
address,"Osborn Rd, McLaren Vale"
locality,McLaren Vale
locality_verbose,McLaren Vale
longitude,138.545242
latitude,-35.198372
cuisines,Modern Australian


# Converter Moedas

In [36]:
df1['currency'].unique()

array(['Botswana Pula(P)', 'Brazilian Real(R$)', 'Dollar($)',
       'Emirati Diram(AED)', 'Indian Rupees(Rs.)',
       'Indonesian Rupiah(IDR)', 'NewZealand($)', 'Pounds(£)',
       'Qatari Rial(QR)', 'Rand(R)', 'Sri Lankan Rupee(LKR)',
       'Turkish Lira(TL)'], dtype=object)

In [6]:
taxas = {
    'Botswana Pula(P)': 0.075,  
    'Brazilian Real(R$)': 0.21,  
    'Dollar($)': 1.0, 
    'Emirati Diram(AED)': 0.27,  
    'Indian Rupees(Rs.)': 0.012,  
    'Indonesian Rupiah(IDR)': 0.000067,  
    'NewZealand($)': 0.62,  
    'Pounds(£)': 1.27,  
    'Qatari Rial(QR)': 0.27,  
    'Rand(R)': 0.054,  
    'Sri Lankan Rupee(LKR)': 0.0032,  
    'Turkish Lira(TL)': 0.42  
}

# Função para converter a moeda para dólar
def converter_para_dolar(moeda, valor):
    if moeda in taxas:
        taxa = taxas[moeda]
        valor_em_dolar = valor * taxa
        return valor_em_dolar
    else:
        return None

In [7]:
df1['average_cost_for_two_dollar'] = df1[['average_cost_for_two', 'currency']].apply(lambda x: converter_para_dolar(x['currency'], x['average_cost_for_two'] ), axis = 1)

In [32]:
df1[df1['average_cost_for_two'] == 25000017.0].T

,385
restaurant_id,16608070
restaurant_name,d'Arry's Verandah Restaurant
country_code,14
city,Adelaide
address,"Osborn Rd, McLaren Vale"
locality,McLaren Vale
locality_verbose,McLaren Vale
longitude,138.545242
latitude,-35.198372
cuisines,Modern Australian


0

## Geral

### 1. Quantos restaurantes únicos estão registrados?

In [18]:
num_restaurantes = df1['restaurant_id'].nunique()
print('A quantidade de restaurantes unico são de : {} Restaurantes' .format(num_restaurantes))

A quantidade de restaurantes unico são de : 6929 Restaurantes


### 2. Quantos países únicos estão registrados?

In [19]:
df1.loc[0,:]
num_pais = df1['country_code'].nunique()
print('A quantidade de paises unicos são de {} paises' .format(num_pais))

A quantidade de paises unicos são de 15 paises


### 3. Quantas cidades únicas estão registradas?

In [20]:
df1.loc[0,:]
num_cidades = df1['city'].nunique()
print('A quantidade de cidades unicas são de {} cidades' .format(num_cidades))
num_cidades

A quantidade de cidades unicas são de 125 cidades


125

### 4. Qual o total de avaliações feitas?

In [19]:
total_votes = df1['votes'].sum()
total_votes

4188336

### 5. Qual o total de tipos de culinária registrados?

In [20]:
df1.loc[0,:]
total_cuisines = df1['cuisines'].nunique()
total_cuisines

163

## Pais

### 1. Qual o nome do país que possui mais cidades registradas?


In [21]:
cols = ['country_name','city','restaurant_id']
aux = df1.loc[:,cols].groupby(['country_name', 'city']).count().reset_index()
aux = aux.loc[:,['country_name', 'city']].groupby('country_name').count().sort_values(by = 'city',ascending=False).reset_index().head()
bar = px.bar(aux, x = 'country_name', y = 'city', title  = 'Quantidade de Restaurante por pais',
             labels={
                 'country_name' : 'Pais',
                 'city': 'Qtd de Cidades'
             })
bar


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Pais=%{x}<br>Qtd de Cidades=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['India', 'United States of America', 'Philippines', 'South Africa',
                          'England'], dtype=object),
              'xaxis': 'x',
              'y': array([49, 22, 12, 12,  5], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Quantidade de Restaurante por pais'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Pais'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Qtd de Cidades'}}}
})

### 2. Qual o nome do país que possui mais restaurantes registrados?


In [ ]:
cols = ['country_name', 'restaurant_id']
aux = df1.loc[:,cols].groupby('country_name').count().sort_values( by= 'restaurant_id',ascending=False).head(10).reset_index().head()
px.bar(aux, x = 'country_name', y = 'restaurant_id',
        title= 'Quantidade de Restaurantes por pais',
        labels= {
            'country_name': 'Pais',
            'restaurant_id': 'Qtd de Restaurantes'
        }
        )

### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?


In [ ]:
df1.loc[0,:]
cols = ['country_name','restaurant_id']
aux3 = df1[df1['price_range'] == 'gourmet']
aux3 = aux3.loc[:,cols].groupby('country_name').count().sort_values(by = 'restaurant_id', ascending= False).reset_index().head(1)
print(aux3.loc[0,'country_name'])

### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [ ]:
cols = ['country_name', 'cuisines']
aux4 = df1.loc[:,cols].groupby(['country_name', 'cuisines']).count().reset_index()
aux4.loc[:,cols].groupby('country_name').count().sort_values(by = 'cuisines', ascending=False).reset_index()

### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?


In [ ]:
df1.loc[0,:]
cols = ['country_name', 'votes']
df1.loc[:,cols].groupby('country_name').sum().sort_values(by = 'votes', ascending= False)

### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [ ]:
df1.loc[0,:]
aux5 = df1[df1['has_online_delivery'] == 1]
cols = ['country_name', 'has_online_delivery']
aux5.loc[:,cols].groupby('country_name').count().sort_values(by = 'has_online_delivery', ascending= False).reset_index()

### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [ ]:
aux6 = df1[df1['has_table_booking'] == 1]
cols = ['country_name', 'has_table_booking']
aux5.loc[:,cols].groupby('country_name').count().sort_values(by = 'has_table_booking', ascending= False).reset_index()

### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [ ]:
cols = ['country_name','votes']
aux = df1.loc[:,cols].groupby('country_name').mean().sort_values(by = 'votes', ascending= False).reset_index()
px.bar(aux, x = 'country_name', y = 'votes',
        title= 'Avaliação média por pais',
        labels= {
            'country_name': 'Pais',
            'votes': 'Média de Avaliações'
        })

### 9. Qual o nome do país que possui, na média, a maior nota média registrada?


In [ ]:
df1.columns
cols = ['country_name', 'aggregate_rating']
aux = df1.loc[:,cols].groupby('country_name').mean('aggregate_rating')
aux = aux.reset_index()
aux = aux.sort_values(by = 'aggregate_rating', ascending= False)
px.bar(aux, x = 'country_name', y = 'aggregate_rating',
        title= 'Avaliação média por pais',
        labels= {
            'country_name': 'Pais',
            'aggregate_rating': 'Avaliação'
        })

### 10. Qual o nome do país que possui, na média, a menor nota média registrada?


In [ ]:
cols = ['country_name', 'aggregate_rating']
aux7 = df1.loc[:,cols].groupby('country_name').mean('aggregate_rating')
aux7 = aux7.reset_index()
aux7.sort_values(by = 'aggregate_rating', ascending= True)

### 11. Qual a média de preço de um prato para dois por país?


In [ ]:
df1.columns
cols = ['country_name', 'average_cost_for_two']
aux = df1[cols].groupby('country_name').mean('average_cost_for_two')
aux = aux.reset_index()
aux = aux.sort_values(by = 'average_cost_for_two', ascending= False).head()
px.bar(aux, x = 'country_name', y = 'average_cost_for_two', 
        title= 'Média de Preço por um prato para duas pessoas',
        labels ={
                'country_name': 'Pais',
                'average_cost_for_two': 'Preço Médio'
        } )

## Cidade

### 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [ ]:
df1.columns
cols = ['restaurant_id', 'city', 'country_name']
aux = df1.loc[:,cols].groupby(['country_name','city']).count().sort_values(by = 'restaurant_id', ascending= False).reset_index().head(10)
px.bar(aux, x = 'city', y = 'restaurant_id', 
       title= 'Top 10 Cidades com maior quantidade de resutaurantes',
       labels= {
           'city': 'Cidade',
           'restaurant_id':  'Qte de restaurantes',
           'country_name': 'Pais'},
       color= 'country_name'
       )

### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [ ]:
cols = ['city','restaurant_id', 'country_name']
aux = df1[df1['aggregate_rating'] > 4]
aux = aux.loc[:,cols].groupby(['country_name','city']).count().sort_values(by = 'restaurant_id', ascending= False).reset_index().head(10)
px.bar(aux, x = 'city', y = 'restaurant_id', 
        title = 'Cidades com restaurantes avaliados acima de 4',
        labels={
            'city': 'Cidade',
            'restaurant_id':'Quantidade de Restaurantes',
            'country_name':'Pais'
        },
        color = 'country_name') 

### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [22]:
cols = ['city','restaurant_id', 'country_name']
aux = df1[df1['aggregate_rating'] < 2.5]
aux = aux.loc[:,cols].groupby(['country_name','city']).count().sort_values(by = 'restaurant_id', ascending= False).reset_index().head(10)
aux.columns = ['country_name','city', 'num_restaurantes']

px.bar(aux, x = 'city', y = 'num_restaurantes', 
        title = 'Cidades com restaurantes avaliados abaixo de 2.5',
        labels={
            'city': 'Cidade',
            'num_restaurantes':'Quantidade de Restaurantes',
            'country_name':'Pais'
        })

In [ ]:
cols = ['country_name','restaurant_id']
aux2 = df1[df1['aggregate_rating'] < 2.5]
aux2 = aux2.loc[:,cols].groupby('country_name').count().sort_values(by = 'country_name', ascending= False).reset_index()
aux2.columns = ['country_name', 'num_restaurantes']

aux3 = df1.loc[:,cols].groupby('country_name').count().sort_values(by = 'country_name', ascending= False).reset_index()
qtd_rest = aux3.restaurant_id
aux2['prop'] =(aux2.num_restaurantes / qtd_rest)*100
aux2 = aux2.sort_values(by = 'prop', ascending= False)
px.bar(aux2, x = 'country_name',y = 'prop',
       title= 'Proporção de Restaurantes com notas abaixo de 2.5 por pais',
       labels={
            'prop': 'Proporção (%)',
            'country_name': 'Pais'
            
       })

### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?


In [ ]:
df1.columns

In [ ]:
cols = ['city', 'average_cost_for_two']
df1.loc[:,cols].groupby('city').mean().sort_values(by = 'average_cost_for_two', ascending= False)

### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [ ]:
df1.columns
cols = ['city', 'cuisines']
aux = df1[cols].groupby('city').nunique().sort_values(by = 'cuisines', ascending= False).reset_index().head(10)
px.bar(aux, x = 'city', y = 'cuisines')

### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [ ]:
df1.columns
aux = df1[df1['has_table_booking'] == 1]
cols = ['city', 'has_table_booking']
aux[cols].groupby('city').count().sort_values(by = 'has_table_booking', ascending= False).reset_index().head(10)

### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [ ]:
df1.columns
aux = df1[df1['is_delivering_now'] == 1]
cols = ['city', 'is_delivering_now']
aux[cols].groupby('city').count().sort_values(by = 'is_delivering_now', ascending= False).reset_index().head(10)

### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [ ]:
df1.columns
aux = df1[df1['has_online_delivery'] == 1]
cols = ['city', 'has_online_delivery']
aux[cols].groupby('city').count().sort_values(by = 'has_online_delivery', ascending= False).reset_index().head(10)

#### MAPA

In [ ]:

import folium
from folium.plugins import MarkerCluster


cols = ['restaurant_name','cuisines','rating_color_name','price_range','longitude','latitude','aggregate_rating', 'average_cost_for_two']
aux = df1[cols]
def mapMaker(df1):
    map = folium.Map(location= [-14,-53], zoom_start= 5)
    make_cluster = MarkerCluster().add_to(map)

    for intex, df in aux.iterrows():
        
        restaurante = df['restaurant_name']
        cuisises = df['cuisines']
        rating = df['aggregate_rating']
        
        html = '<p><strong> {} </strong></p>'
        html += '<p>{}</p>'
        html += '<p>{}</p>'
        pp = folium.Html(html.format(restaurante,cuisises,rating),script= True)
        folium.Marker(location= [df['latitude'],df['longitude']],
                        icon= folium.Icon(color = df['rating_color_name']),
                        popup= folium.Popup(pp, max_width=500)
                        ).add_to(make_cluster)


## Restaurantes

### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [ ]:
cols = ['restaurant_name', 'country_name', 'votes']
aux = df1[cols].sort_values(by = 'votes', ascending= False).head(1)
print('O restaurante com a maior quantidade de votos é {}'.format(aux.iloc[0,0]))

In [ ]:
df1[cols].groupby('restaurant_name').sum('votes').sort_values(by = 'votes', ascending= False).reset_index().head(10)

In [ ]:
df1.restaurant_id.nunique()

In [ ]:
len(df1)

### 2. Qual o nome do restaurante com a maior nota média?

In [ ]:
df1.columns
cols = ['restaurant_id','country_name', 'aggregate_rating']
df1[cols].sort_values(by = ['aggregate_rating'], ascending= False).head(10)

### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [ ]:
df1.columns
cols = ['restaurant_name', 'country_name','average_cost_for_two']
df1[cols].sort_values(by = 'average_cost_for_two', ascending= False).head(10)

### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [ ]:
df1.columns
aux = df1[df1['cuisines'] == 'Brazilian']
cols = ['restaurant_id','country_name','aggregate_rating' ]
aux[cols].sort_values(by = 'aggregate_rating').head(10)

### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [ ]:
df1.columns
aux = df1[(df1['cuisines'] == 'Brazilian') & (df1['country_name'] == 'Brazil')]
cols = ['restaurant_id', 'restaurant_name','aggregate_rating' ]
aux[cols].sort_values(by = 'aggregate_rating', ascending= False).head(10)

### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [ ]:
df1.columns

cols = ['restaurant_name', 'has_online_delivery','votes']
aux = df1[cols].groupby('has_online_delivery').mean('votes').reset_index()
aux.columns = ['Faz Pedidos Online', 'Votos']
aux['Faz Pedidos Online']= aux['Faz Pedidos Online'].map({0: 'Não', 1: 'Sim'})
aux

### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [ ]:
df1.columns
cols = ['has_table_booking', 'average_cost_for_two']
aux = df1[cols].groupby('has_table_booking').mean('average_cost_for_two').reset_index()
aux['has_table_booking'] = aux['has_table_booking'].map({0: 'Não', 1: 'Sim'})
aux

###  8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?


In [ ]:
df1.columns
cuisines = ['Japanese', 'BBQ']
aux = df1[(df1['cuisines'].isin(cuisines)) & (df1['country_name'] == 'United States of America' )]
cols = ['cuisines', 'average_cost_for_two' ]
aux[cols].groupby('cuisines').mean('average_cost_for_two').reset_index()

## Tipos de Culinária

### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [12]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'country_name', 'rating_color_name'],
      dtype='object')

In [11]:
cuisines_ratting(df = df1, cuisine= 'Italian', ascending= False).head(1)

,restaurant_name,country_name,aggregate_rating
366,Andre's Cucina & Polenta Bar,Australia,4.9


### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [32]:
cuisines_ratting(df = df1, cuisine= 'Italian').head(1)

,restaurant_name,country_name,aggregate_rating
6723,Le Delicatezze Di Bruno,England,0.0


### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [33]:
cuisines_ratting(df = df1, cuisine= 'American', ascending= False).head(1)

,restaurant_name,country_name,aggregate_rating
726,OEB Breakfast Co.,Canada,4.9


### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [10]:
cuisines_ratting(df = df1, cuisine= 'American').head(1)

,restaurant_name,country_name,aggregate_rating
6751,Alston Bar & Beef,England,0.0


### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
cuisines_ratting(df = df1, cuisine= 'Arabian', ascending= False).head()

### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
cuisines_ratting(df = df1, cuisine= 'Arabian').head()

### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
cuisines_ratting(df = df1, cuisine= 'Japanese', ascending= False).head()

### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
cuisines_ratting(df = df1, cuisine= 'Japanese').head()

### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
cuisines_ratting(df = df1, cuisine= 'Home-made', ascending= False)

### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
cuisines_ratting(df = df1, cuisine= 'Home-made')

### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [ ]:
cols = ['cuisines', 'average_cost_for_two']
df1[cols].groupby('cuisines').mean().sort_values('average_cost_for_two', ascending= False)

### 12. Qual o tipo de culinária que possui a maior nota média?

In [15]:
filtro = 11
cols = ['cuisines', 'aggregate_rating']
aux = df1[cols].groupby('cuisines').mean('aggregate_rating').sort_values('aggregate_rating', ascending = False).reset_index().head(filtro)

px.bar(aux, x = 'cuisines', y ='aggregate_rating',
        labels= {
            'aggregate_rating': 'Avaliação',
            'cuisines': 'Culinária'
        },
        title= 'Top ' + str(filtro) + ' melhores tipos de culinárias'
        )

In [16]:
filtro = 11
ols = ['cuisines', 'aggregate_rating']
aux = df1[cols].groupby('cuisines').mean('aggregate_rating').sort_values('aggregate_rating').reset_index().head(filtro)

px.bar(aux, x = 'cuisines', y ='aggregate_rating',
        labels= {
            'aggregate_rating': 'Avaliação',
            'cuisines': 'Culinária'
        },
        title= 'Top ' + str(filtro) + ' piores tipos de culinárias'
        )

### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [ ]:
cols = ['cuisines', 'restaurant_id']
aux = df1[(df1['has_online_delivery'] == 1) & (df1['is_delivering_now'] == 1)]
aux[cols].groupby('cuisines').count().sort_values(by = 'restaurant_id', ascending= False).reset_index()


### TOP Restaurantes

In [ ]:
filtro = 10
cols = df1.columns[[0,1,21,3,9,10,17,20]]

aux = df1[cols].sort_values(by = ['aggregate_rating', 'votes'], ascending= False).head(filtro)
aux
